# Análisis por Período

#### Condiciones Iniciales:
 - Los Relevamientos se obtienen en varios csv
 - Debido al tiempo que se requiere para correr el scrapping puede que haya dos fechas para el mismo período (ponemos a correr el scrapper el 28 a la noche y termina el 29 a la mañana)
 - Los datos se guardan en una carpeta nombrada como le fecha del relevamiento (Ej: 20180928)
 - Internamente se guardan diversos csv que provienen de la estructura de scrapping que tenemos, todos cnon la misma estructura
 
#### Objetivo:

Un notebook que, a partir de fecha_inicio y fecha_fin, lea las carpetas de dicha fecha, levante los csv y realice comparativas de precios desde el inicio del periodo al fin del mismo.

#### Parámetros

In [1]:
fecha_inicio_relevamiento = "20200413" # Las fecha debe coincidir con nombres de carpetas
fecha_fin_relevamiento = "20200419" # Las fecha debe coincidir con nombres de carpetas

provincia_relevamiento = "AR-X" # Córdoba

sup_titulo_relevamiento = "Precios Marzo 20"
titulo_relevamiento = "Relevamiento del Programa Precios Claros hecho por @opendatacba, para la provincia de Córdoba."


In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10,6)

In [3]:
def obtener_path_archivos_de_precios(fecha):
    """Funcion para obtener el listado de archivos csv que almacenan los precios para el relevamiento
    hecho en la fecha pasada por parámetro"""
    
    files_to_read = []
    root_precios = os.path.join(".","datasets","precios", fecha)
    for root, dirs, files in os.walk(root_precios):
        for fname in files:
            files_to_read.append(os.path.join(root, fname))
            
    return(files_to_read)

In [4]:
def obtener_dataset_relevamiento(fecha, inicio_periodo):
    """Funcion que lee la carpeta correspondiente y devuelve un dataframe 
    conteniendo los datos del producto, del establecimiento relevado, el precio
    relevado y la fecha del relevamiento"""
        
    path_archivos_datos = obtener_path_archivos_de_precios(fecha)
    columnas_a_leer = ["precio", "producto_id", "sucursal_id"]

    precios = pd.concat((pd.read_csv(f, low_memory=False, usecols=columnas_a_leer) for f in path_archivos_datos))

    precios = precios[precios["precio"].notnull()]

    # Piso la fecha en la cual se ejecuto el scrapper por la fecha del relevamiento (ver consideraciones)
    precios["fecha_relevamiento"] = pd.to_datetime(fecha)
    
    if inicio_periodo:
        precios.rename({"fecha_relevamiento":"fecha_relevamiento_inicio_periodo","precio": "precio_inicio_periodo"}, axis="columns", inplace=True)
    else:
        precios.rename({"fecha_relevamiento":"fecha_relevamiento_fin_periodo","precio": "precio_fin_periodo"}, axis="columns", inplace=True)   
    
    return(precios)

## Armar dataset comparativo para el Período

#### Cargar datasets de precios y comparar periodos

In [5]:
df_inicio = obtener_dataset_relevamiento(fecha_inicio_relevamiento, inicio_periodo=True)
df_fin = obtener_dataset_relevamiento(fecha_fin_relevamiento, inicio_periodo=False)

df_precios = df_inicio.merge(df_fin, left_on=["producto_id", "sucursal_id"], right_on=["producto_id", "sucursal_id"])
df_precios = df_precios[["producto_id", "sucursal_id", "precio_inicio_periodo", "precio_fin_periodo"]]
df_precios["variacion_periodo"] = round((df_precios["precio_fin_periodo"] / df_precios["precio_inicio_periodo"]) - 1,2)
df_precios.head()

,producto_id,sucursal_id,precio_inicio_periodo,precio_fin_periodo,variacion_periodo
0,0000000221184,9-2-583,259.0,259.0,0.0
1,0000000221184,9-2-953,439.0,439.0,0.0
2,0000000231688,9-2-583,69.9,69.9,0.0
3,0000000231688,9-2-953,110.0,110.0,0.0
4,0000000288989,9-2-583,394.9,394.9,0.0


#### Merge con datasets de Productos

In [6]:
productos = pd.read_csv("./datasets/productos.csv")
productos = productos.drop_duplicates() # El csv tiene muchos duplicados
df_precios = df_precios.merge(productos, left_on="producto_id", right_on="id")

df_precios["nombre"] = df_precios["nombre"].astype("category")
df_precios["marca"] = df_precios["marca"].astype("category")
df_precios["presentacion"] = df_precios["presentacion"].astype("category")
df_precios.drop("id", axis="columns", inplace=True)
df_precios.head()

,producto_id,sucursal_id,precio_inicio_periodo,precio_fin_periodo,variacion_periodo,marca,nombre,presentacion
0,0000000221184,9-2-583,259.0,259.0,0.0,PALADINI,Salchichon Feteado Paladini 1 Kg,1.0 kg
1,0000000221184,9-2-953,439.0,439.0,0.0,PALADINI,Salchichon Feteado Paladini 1 Kg,1.0 kg
2,0000000231688,9-2-583,69.9,69.9,0.0,DULCOR,Dulce de Zapallo Dulcor 1 Kg,1.0 kg
3,0000000231688,9-2-953,110.0,110.0,0.0,DULCOR,Dulce de Zapallo Dulcor 1 Kg,1.0 kg
4,0000000288989,9-2-583,394.9,394.9,0.0,CARNEVALLI,Chorizo de Cerdo Carnevalli 1 Kg,1.0 kg


#### Merge con datasets de Sucursales y filtrar por Córdoba

In [7]:
sucursales = pd.read_csv("./datasets/sucursales.csv")
sucursales['sucursal_id'] = sucursales.apply(lambda x: f"{x['banderaId']}-{x['comercioId']}-{x['provincia']}", axis=1)

df_precios = df_precios.merge(sucursales, left_on='sucursal_id', right_on='id')
df_precios.drop(['sucursal_id_y', 'sucursalTipo', 'comercioId', 'banderaId', 'lat', 'lng', 'direccion', 'id'], axis="columns", inplace=True)
df_precios.head()

,producto_id,sucursal_id_x,precio_inicio_periodo,precio_fin_periodo,variacion_periodo,marca,nombre,presentacion,banderaDescripcion,comercioRazonSocial,provincia,localidad,sucursalNombre
0,0000000221184,9-2-583,259.0,259.0,0.0,PALADINI,Salchichon Feteado Paladini 1 Kg,1.0 kg,Disco,Jumbo Retail Argentina S.A.,AR-X,VILLA ALLENDE,SM 583 - Villa Allende.
1,0000000231688,9-2-583,69.9,69.9,0.0,DULCOR,Dulce de Zapallo Dulcor 1 Kg,1.0 kg,Disco,Jumbo Retail Argentina S.A.,AR-X,VILLA ALLENDE,SM 583 - Villa Allende.
2,0000000288989,9-2-583,394.9,394.9,0.0,CARNEVALLI,Chorizo de Cerdo Carnevalli 1 Kg,1.0 kg,Disco,Jumbo Retail Argentina S.A.,AR-X,VILLA ALLENDE,SM 583 - Villa Allende.
3,0000000289153,9-2-583,239.9,239.9,0.0,CARNEVALLI,Morcilla con Cebolla Carnevalli 1 Kg,1.0 kg,Disco,Jumbo Retail Argentina S.A.,AR-X,VILLA ALLENDE,SM 583 - Villa Allende.
4,0000000289177,9-2-583,489.9,489.9,0.0,CARNEVALLI,Salchichas Fresca Carnevalli 1 Kg,1.0 kg,Disco,Jumbo Retail Argentina S.A.,AR-X,VILLA ALLENDE,SM 583 - Villa Allende.


#### Crear nuevo DF para Córdoba

In [8]:
df_cba = df_precios[df_precios.provincia == provincia_relevamiento]
df_cba.head()

,producto_id,sucursal_id_x,precio_inicio_periodo,precio_fin_periodo,variacion_periodo,marca,nombre,presentacion,banderaDescripcion,comercioRazonSocial,provincia,localidad,sucursalNombre
0,0000000221184,9-2-583,259.0,259.0,0.0,PALADINI,Salchichon Feteado Paladini 1 Kg,1.0 kg,Disco,Jumbo Retail Argentina S.A.,AR-X,VILLA ALLENDE,SM 583 - Villa Allende.
1,0000000231688,9-2-583,69.9,69.9,0.0,DULCOR,Dulce de Zapallo Dulcor 1 Kg,1.0 kg,Disco,Jumbo Retail Argentina S.A.,AR-X,VILLA ALLENDE,SM 583 - Villa Allende.
2,0000000288989,9-2-583,394.9,394.9,0.0,CARNEVALLI,Chorizo de Cerdo Carnevalli 1 Kg,1.0 kg,Disco,Jumbo Retail Argentina S.A.,AR-X,VILLA ALLENDE,SM 583 - Villa Allende.
3,0000000289153,9-2-583,239.9,239.9,0.0,CARNEVALLI,Morcilla con Cebolla Carnevalli 1 Kg,1.0 kg,Disco,Jumbo Retail Argentina S.A.,AR-X,VILLA ALLENDE,SM 583 - Villa Allende.
4,0000000289177,9-2-583,489.9,489.9,0.0,CARNEVALLI,Salchichas Fresca Carnevalli 1 Kg,1.0 kg,Disco,Jumbo Retail Argentina S.A.,AR-X,VILLA ALLENDE,SM 583 - Villa Allende.


In [141]:
df_precios[['marca', 'nombre', 'producto_id', 'precio_fin_periodo', 'comercioRazonSocial', 'localidad']] \
    .query('producto_id == "7702018037803"')

,marca,nombre,producto_id,precio_fin_periodo,comercioRazonSocial,localidad
453,GILLETTE,Rasuradora Recargable Mach3 Sensitibe Gillette...,7702018037803,519.00,Jumbo Retail Argentina S.A.,VILLA ALLENDE
27715,GILLETTE,Rasuradora Recargable Mach3 Sensitibe Gillette...,7702018037803,418.00,S.A. Importadora y Exportadora de la Patagonia,Villa Regina
30159,GILLETTE,Rasuradora Recargable Mach3 Sensitibe Gillette...,7702018037803,429.00,S.A. Importadora y Exportadora de la Patagonia,Zapala
32653,GILLETTE,Rasuradora Recargable Mach3 Sensitibe Gillette...,7702018037803,404.25,Alberdi S.A.,Salta
39547,GILLETTE,Rasuradora Recargable Mach3 Sensitibe Gillette...,7702018037803,404.25,Alberdi S.A.,San Ramon de la Nueva Oran
46816,GILLETTE,Rasuradora Recargable Mach3 Sensitibe Gillette...,7702018037803,419.65,Cooperativa Agrícola Ganadera e Industrial de ...,Carmen de Patagones
54216,GILLETTE,Rasuradora Recargable Mach3 Sensitibe Gillette...,7702018037803,519.00,Jumbo Retail Argentina S.A.,NEUQUEN
137949,GILLETTE,Rasuradora Recargable Mach3 Sensitibe Gillette...,7702018037803,384.00,INC S.A.,Córdoba
179878,GILLETTE,Rasuradora Recargable Mach3 Sensitibe Gillette...,7702018037803,479.00,Jumbo Retail Argentina S.A.,GENERAL ROCA


# Analisis para la Provincia de Córdoba

In [35]:
columnas_tabla = ['marca', 'producto_id', 'nombre', 'precio_inicio_periodo', 'precio_fin_periodo', 'variacion_periodo', 'comercioRazonSocial', 'localidad', 'provincia']
columnas_tabla_dispersion = ['producto_id', 'nombre', 'precio_fin_periodo', 'comercioRazonSocial', 'localidad', 'provincia']

In [11]:
# Sacamos outliers
# df_precios = df_precios[(df_precios.variacion_periodo > -0.20) & (df_precios.variacion_periodo < 0.30)]

In [70]:
#ax = df_precios.groupby(["empresa","sucursalNombre"])["variacion_periodo"].mean().sort_values().plot(kind="barh")
#vals = ax.get_xticks()
#ax.set_xticklabels(['{:,.2%}'.format(x) for x in vals])
#plt.suptitle(sup_titulo_relevamiento, size=16)
#plt.title(titulo_relevamiento, size=10)
#plt.ylabel("")
#plt.xlabel("Aumento Promedio de Precios", size=14)
#plt.show()

### 20 Productos que más aumentaron en la Provincia

In [26]:
top_20_aumentos = df_cba.sort_values(ascending=False, by="variacion_periodo").head(20)
top_20_aumentos[columnas_tabla]

,marca,producto_id,nombre,precio_inicio_periodo,precio_fin_periodo,variacion_periodo,comercioRazonSocial,localidad,provincia
137949,GILLETTE,7702018037803,Rasuradora Recargable Mach3 Sensitibe Gillette...,179.00,384.00,1.15,INC S.A.,Córdoba,AR-X
251,MAGISTRAL,7506309814629,Detergente Aloe Magistral 500 Cc,79.00,169.00,1.14,Jumbo Retail Argentina S.A.,VILLA ALLENDE,AR-X
303,MAGISTRAL,7506339398953,Detergente Lavavajilla Botella Magistral 500 Cc,79.00,169.00,1.14,Jumbo Retail Argentina S.A.,VILLA ALLENDE,AR-X
257,MAGISTRAL,7506309878997,Detergente Manzana Verde Magistral 500 Ml,79.00,149.00,0.89,Jumbo Retail Argentina S.A.,VILLA ALLENDE,AR-X
2842,VILLA DEL SUR,7798062541397,Agua Mineral sin Gas en Bidon Villa del Sur 6....,108.00,180.00,0.67,Jumbo Retail Argentina S.A.,VILLA ALLENDE,AR-X
10702,NUTRILON,7795323003519,Alimento 5 Cereales sin Azucar Nutrilon 150 Gr,71.40,119.00,0.67,INC S.A.,Río Cuarto,AR-X
2838,VILLA DEL SUR,7798062540116,Agua Mineral sin Gas en Bidon Villa del Sur 6....,108.00,180.00,0.67,Jumbo Retail Argentina S.A.,VILLA ALLENDE,AR-X
138787,NUGATÓN,7792360091974,Oblea de Chocolate Nugaton 8.5 Gr,10.20,17.00,0.67,INC S.A.,Córdoba,AR-X
2258,JARDÍN,7792389000643,Limpiador Liquido Cristal Jardin Arpege 900 Cc,49.00,72.00,0.47,Jumbo Retail Argentina S.A.,VILLA ALLENDE,AR-X
2260,JARDÍN,7792389000704,Limpiador Liquido Colonia Cremo Jardin 900 Cc,49.00,72.00,0.47,Jumbo Retail Argentina S.A.,VILLA ALLENDE,AR-X


### Qué onda la Gillete?

Vamos a ver que onda el precio de la Gillete que aumentó un 115%, se trata de este solo supermercado?

In [27]:
df_precios[columnas_tabla].query('producto_id == "7702018037803"')

,marca,producto_id,nombre,precio_inicio_periodo,precio_fin_periodo,variacion_periodo,comercioRazonSocial,localidad,provincia
453,GILLETTE,7702018037803,Rasuradora Recargable Mach3 Sensitibe Gillette...,519.00,519.00,0.00,Jumbo Retail Argentina S.A.,VILLA ALLENDE,AR-X
27715,GILLETTE,7702018037803,Rasuradora Recargable Mach3 Sensitibe Gillette...,418.00,418.00,0.00,S.A. Importadora y Exportadora de la Patagonia,Villa Regina,AR-R
30159,GILLETTE,7702018037803,Rasuradora Recargable Mach3 Sensitibe Gillette...,429.00,429.00,0.00,S.A. Importadora y Exportadora de la Patagonia,Zapala,AR-Q
32653,GILLETTE,7702018037803,Rasuradora Recargable Mach3 Sensitibe Gillette...,404.25,404.25,0.00,Alberdi S.A.,Salta,AR-A
39547,GILLETTE,7702018037803,Rasuradora Recargable Mach3 Sensitibe Gillette...,404.25,404.25,0.00,Alberdi S.A.,San Ramon de la Nueva Oran,AR-A
46816,GILLETTE,7702018037803,Rasuradora Recargable Mach3 Sensitibe Gillette...,419.65,419.65,0.00,Cooperativa Agrícola Ganadera e Industrial de ...,Carmen de Patagones,AR-B
54216,GILLETTE,7702018037803,Rasuradora Recargable Mach3 Sensitibe Gillette...,519.00,519.00,0.00,Jumbo Retail Argentina S.A.,NEUQUEN,AR-Q
137949,GILLETTE,7702018037803,Rasuradora Recargable Mach3 Sensitibe Gillette...,179.00,384.00,1.15,INC S.A.,Córdoba,AR-X
179878,GILLETTE,7702018037803,Rasuradora Recargable Mach3 Sensitibe Gillette...,479.00,479.00,0.00,Jumbo Retail Argentina S.A.,GENERAL ROCA,AR-R


Vemos que la dispersión en los precios de la Gillette es enorme de supermercado en supermercado. Va desde los 404.25 en Salta a 519 en Neuquén o Villa Allende

### Pasa lo mismo con los detergentes o las aguas?

In [31]:
df_precios[columnas_tabla].query('producto_id == "7506309814629"')

,marca,producto_id,nombre,precio_inicio_periodo,precio_fin_periodo,variacion_periodo,comercioRazonSocial,localidad,provincia
251,MAGISTRAL,7506309814629,Detergente Aloe Magistral 500 Cc,79.0,169.0,1.14,Jumbo Retail Argentina S.A.,VILLA ALLENDE,AR-X
12606,MAGISTRAL,7506309814629,Detergente Aloe Magistral 500 Cc,131.0,131.0,0.00,Jumbo Retail Argentina S.A.,MARTINEZ,AR-B
70038,MAGISTRAL,7506309814629,Detergente Aloe Magistral 500 Cc,129.0,129.0,0.00,California S.A.,Posadas,AR-N
132482,MAGISTRAL,7506309814629,Detergente Aloe Magistral 500 Cc,151.0,151.0,0.00,Jumbo Retail Argentina S.A.,COMODORO RIVADAVIA,AR-U
172260,MAGISTRAL,7506309814629,Detergente Aloe Magistral 500 Cc,123.0,123.0,0.00,Jumbo Retail Argentina S.A.,ZONA DELTA TIGRE,AR-B
184550,MAGISTRAL,7506309814629,Detergente Aloe Magistral 500 Cc,118.0,118.0,0.00,Jumbo Retail Argentina S.A.,VILLA GESELL,AR-B


Porqué el mismo detergente se vende a 118 en Villa Gessel y a 169 en Villa Allende? un 43% de Diferencia
Porqué el mismo detergente se vende a 123 en Zona Delta Tigre y a 151 en Comodoro Rivadavia? un 22% de Diferencia (Ponele que Patagonia)
Y todos de la misma cadena! (Jumbo Retail Argentina)

In [29]:
df_precios[columnas_tabla].query('producto_id == "7798062541397"').sort_values(by='precio_fin_periodo', ascending=False)

,marca,producto_id,nombre,precio_inicio_periodo,precio_fin_periodo,variacion_periodo,comercioRazonSocial,localidad,provincia
31272,VILLA DEL SUR,7798062541397,Agua Mineral sin Gas en Bidon Villa del Sur 6....,222.00,222.00,0.00,S.A. Importadora y Exportadora de la Patagonia,Zapala,AR-Q
91899,VILLA DEL SUR,7798062541397,Agua Mineral sin Gas en Bidon Villa del Sur 6....,222.00,222.00,0.00,INC S.A.,Río Grande,AR-V
155354,VILLA DEL SUR,7798062541397,Agua Mineral sin Gas en Bidon Villa del Sur 6....,221.00,221.00,0.00,Almacenes Pampas S.A.,BERAZATEGUI,AR-B
164170,VILLA DEL SUR,7798062541397,Agua Mineral sin Gas en Bidon Villa del Sur 6....,220.00,220.00,0.00,Almacenes de Marca S.A.,LA PLATA,AR-B
146537,VILLA DEL SUR,7798062541397,Agua Mineral sin Gas en Bidon Villa del Sur 6....,220.00,220.00,0.00,Supermercado El Nene S.A.,LA PLATA,AR-B
29518,VILLA DEL SUR,7798062541397,Agua Mineral sin Gas en Bidon Villa del Sur 6....,217.00,217.00,0.00,S.A. Importadora y Exportadora de la Patagonia,Villa Regina,AR-R
131364,VILLA DEL SUR,7798062541397,Agua Mineral sin Gas en Bidon Villa del Sur 6....,211.00,211.00,0.00,INC S.A.,Gualeguaychu,AR-E
117665,VILLA DEL SUR,7798062541397,Agua Mineral sin Gas en Bidon Villa del Sur 6....,210.49,210.49,0.00,Wal Mart Argentina S.R.L.,COMODORO RIVADAVIA,AR-U
7744,VILLA DEL SUR,7798062541397,Agua Mineral sin Gas en Bidon Villa del Sur 6....,199.00,205.95,0.03,Libertad S.A,Rafaela,AR-S
7745,VILLA DEL SUR,7798062541397,Agua Mineral sin Gas en Bidon Villa del Sur 6....,199.00,199.00,0.00,Libertad S.A,Rafaela,AR-S


Un Bidon de agua de 6 Litros en LA PLATA varia de 114.99 a 220. Casi el doble.
Walmart Argentina en Corrientes vende el Bidon a 155.79 y el Libertad de Rafaela a 205. Un 30% de Diferencia

In [36]:
df_cba[columnas_tabla_dispersion].query('producto_id == "7798062541397"').sort_values(by='precio_fin_periodo', ascending=False)

,producto_id,nombre,precio_fin_periodo,comercioRazonSocial,localidad,provincia
2842,7798062541397,Agua Mineral sin Gas en Bidon Villa del Sur 6....,180.00,Jumbo Retail Argentina S.A.,VILLA ALLENDE,AR-X
76504,7798062541397,Agua Mineral sin Gas en Bidon Villa del Sur 6....,173.20,Libertad S.A,Villa Maria,AR-X
139060,7798062541397,Agua Mineral sin Gas en Bidon Villa del Sur 6....,170.00,INC S.A.,Córdoba,AR-X
178367,7798062541397,Agua Mineral sin Gas en Bidon Villa del Sur 6....,161.49,Cyre S.A.,VILLA DEL ROSARIO,AR-X


### Limpiador líquido en Salta

In [37]:
df_precios[columnas_tabla_dispersion].query('producto_id == "7792389000704" and provincia == "AR-A"').sort_values(by='precio_fin_periodo', ascending=False)

,producto_id,nombre,precio_fin_periodo,comercioRazonSocial,localidad,provincia
81155,7792389000704,Limpiador Liquido Colonia Cremo Jardin 900 Cc,74.50,Jumbo Retail Argentina S.A.,SALTA,AR-A
182538,7792389000704,Limpiador Liquido Colonia Cremo Jardin 900 Cc,61.80,Wal Mart Argentina S.R.L.,SALTA,AR-A
107327,7792389000704,Limpiador Liquido Colonia Cremo Jardin 900 Cc,53.00,INC S.A.,Salta,AR-A
36123,7792389000704,Limpiador Liquido Colonia Cremo Jardin 900 Cc,49.99,Alberdi S.A.,Salta,AR-A
43017,7792389000704,Limpiador Liquido Colonia Cremo Jardin 900 Cc,49.99,Alberdi S.A.,San Ramon de la Nueva Oran,AR-A


In [38]:
df_precios[columnas_tabla_dispersion].query('producto_id == "7795697431345"').sort_values(by='precio_fin_periodo', ascending=False)

,producto_id,nombre,precio_fin_periodo,comercioRazonSocial,localidad,provincia
55556,7795697431345,Cerveza Rubia Miller 1 Lt,155.00,Jumbo Retail Argentina S.A.,NEUQUEN,AR-Q
192828,7795697431345,Cerveza Rubia Miller 1 Lt,152.00,Asociación Mutual Socios y Adherentes de la Co...,Capitán Sarmiento,AR-B
2724,7795697431345,Cerveza Rubia Miller 1 Lt,140.00,Jumbo Retail Argentina S.A.,VILLA ALLENDE,AR-X
23186,7795697431345,Cerveza Rubia Miller 1 Lt,112.99,Autoservicio La Amistad Cooperativa de Trabajo...,LA PLATA,AR-B
73456,7795697431345,Cerveza Rubia Miller 1 Lt,109.00,California S.A.,Posadas,AR-N


### Canasta Básica

In [47]:
prod_canasta_basica = [ "7790272001029","7790040872202","7791110001874","7791120031557","7792360002079","7794000597525","7792798000708","7792180001528","7792798006199","7791290011762","7791290012387","7790070411716","7795184005004","7790060023684","7792710000014","7792878000017","7792180004512","7622300847234","7793913000986","7798159715731","7790170901872","7798003690252","7790639001976","7791100000481","7793360801709","7794000597730","7798031154221","7791905002512","7791130052733","7791337120204","7791293030937","7791290012615","7798038152749","7796039001608","7791813423119","7506295302292","7792540250450","7791293018522","7501065904522","7790070410603","7790070318596","7791293030777","7790895000997","7792798007387","2400592000002","7500435012355","7790520018649","7793147000899","7790150006122","7790742163905","7791290001008","7792798007424","7790070507235","7790387113310","7793704000225","7622300724269","7790250019565"]
df_canasta = df_precios[df_precios.producto_id.isin(prod_canasta_basica)]
df_canasta[columnas_tabla].sort_values(by='variacion_periodo', ascending=False).head(20)

,marca,producto_id,nombre,precio_inicio_periodo,precio_fin_periodo,variacion_periodo,comercioRazonSocial,localidad,provincia
65274,COCA COLA,7790895000997,Coca Cola Sabor Original 2.25 Lt,120.00,149.95,0.25,Libertad S.A,Capital,AR-A
182380,COCA COLA,7790895000997,Coca Cola Sabor Original 2.25 Lt,120.00,134.68,0.12,Wal Mart Argentina S.R.L.,SALTA,AR-A
20801,CAÑUELAS,7792180001528,Harina de Trigo 000 Cañuelas 1 Kg,34.99,38.79,0.11,Autoservicio La Amistad Cooperativa de Trabajo...,LA PLATA,AR-B
62043,COCA COLA,7790895000997,Coca Cola Sabor Original 2.25 Lt,125.00,136.00,0.09,Wal Mart Argentina S.R.L.,NEUQUEN,AR-Q
131636,TERRABUSI,7622300724269,Fideos Mostacholes Rayados Terrabusi 500 Gr,50.00,53.99,0.08,Libertad S.A,San Juan,AR-J
15647,TERRABUSI,7622300724269,Fideos Mostacholes Rayados Terrabusi 500 Gr,43.99,46.99,0.07,Autoservicio La Amistad Cooperativa de Trabajo...,LA PLATA,AR-B
167937,SEDAL,7791293030937,Acondicionador Ceramidas Sedal 340 Ml,133.15,142.99,0.07,Alberdi S.A.,San Miguel de Tucumán,AR-T
167930,SEDAL,7791293030777,Shampoo Ceramidas Sedal 340 Ml,133.15,142.99,0.07,Alberdi S.A.,San Miguel de Tucumán,AR-T
178125,PLAYADITO,7793704000225,Yerba Mate con Palo Playadito 1 Kg,219.99,233.69,0.06,Cyre S.A.,VILLA DEL ROSARIO,AR-X
177928,CAÑUELAS,7792180001528,Harina de Trigo 000 Cañuelas 1 Kg,31.99,32.99,0.03,Cyre S.A.,VILLA DEL ROSARIO,AR-X


In [56]:
df_maximos_cba = pd.read_csv('./datasets/precios_maximos_cordoba.csv')
df_maximos_cba.drop(['Producto', 'Producto_s_tilde', 'Provincia', 'Region', 'marca', 'subcategoria'], axis="columns", inplace=True)
df_maximos_cba.head()

,Precio sugerido,categoria,id_producto
0,251.0,Cuidado del Bebe,7790010959797
1,151.0,Aceites,7792180001665
2,89.0,Aceites,7792180001641
3,139.0,Aceites,7790060023684
4,95.0,Aceites,7790070012050


In [58]:
df_maximos = df_cba.merge(df_maximos_cba, left_on='producto_id', right_on='id_producto')
df_maximos.head()

,producto_id,sucursal_id_x,precio_inicio_periodo,precio_fin_periodo,variacion_periodo,marca,nombre,presentacion,banderaDescripcion,comercioRazonSocial,provincia,localidad,sucursalNombre,Precio sugerido,categoria,id_producto
0,3014260273903,9-2-583,299.0,299.0,0.0,GILLETTE,Espuma de Afeitar Regular en Aerosol Foamy 322 Ml,322.0 ml,Disco,Jumbo Retail Argentina S.A.,AR-X,VILLA ALLENDE,SM 583 - Villa Allende.,294.0,Cuidado Facial,3014260273903
1,3014260273903,10-2-119,269.0,269.0,0.0,GILLETTE,Espuma de Afeitar Regular en Aerosol Foamy 322 Ml,322.0 ml,Market,INC S.A.,AR-X,Río Cuarto,Río Cuarto,294.0,Cuidado Facial,3014260273903
2,3014260275143,9-2-583,519.0,519.0,0.0,GILLETTE,Cartuchos Gillette Mach 3 Turbo 2 Un,2.0 un,Disco,Jumbo Retail Argentina S.A.,AR-X,VILLA ALLENDE,SM 583 - Villa Allende.,515.0,Cuidado Facial,3014260275143
3,4005808257553,9-2-583,138.0,138.0,0.0,NIVEA,Desodorante Antitranspirante Hombre en Roll On...,50.0 cc,Disco,Jumbo Retail Argentina S.A.,AR-X,VILLA ALLENDE,SM 583 - Villa Allende.,138.0,Cuidado Corporal,4005808257553
4,4005808257553,10-2-119,121.0,121.0,0.0,NIVEA,Desodorante Antitranspirante Hombre en Roll On...,50.0 cc,Market,INC S.A.,AR-X,Río Cuarto,Río Cuarto,138.0,Cuidado Corporal,4005808257553
